<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/Multiclass_Classification_assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import sys 
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

In [30]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install -U swifter -qq
  !pip install -U nltk -qq
  !pip install -U spacy -qq
  !python -m spacy download en_core_web_sm -qq
  !pip install torchtext --upgrade
  base_path = '/content/drive/MyDrive/NLP/assignment5'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-04-02 13:48:22.304764: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 13:48:24.373891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-02 13:48:27.841793: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.1 MB/s eta 0:00:00
✔ Download and installation successf

In [31]:
from pathlib import Path
import pandas as pd
import torch
import torch.nn as inn
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader


In [32]:
data_folder = Path(base_path)
for entries in data_folder.iterdir():
  print(entries.name)

multiclass_hw_cleaned.csv
Multiclass_Classification_assignment5.ipynb


In [33]:
stackExchange1 = pd.read_csv(data_folder/ 'multiclass_hw_cleaned.csv', encoding = 'ISO-8859-1')
print(stackExchange1.shape)
print(stackExchange1.head(2))

(188878, 6)
   Unnamed: 0                                     Title  \
0           0   detail disclosure indicator on UIButton   
1           1  hello world fails to show up in emulator   

                                                Body  \
0  <p>Is there a simple way to place a detail dis...   
1  <p>I followed Hello World tutorial exactly.  E...   

                                        cleaned_text     Tags  \
0  detail disclosure indicator uibutton simple wa...   iphone   
1  hello world fail emulator follow hello world t...  android   

   Tag_Number_final  
0                 8  
1                 4  


In [34]:
stackExchange = pd.DataFrame().assign(cleaned_text=stackExchange1['cleaned_text'], Tag_Number_final=stackExchange1['Tag_Number_final'])
print(stackExchange)

                                             cleaned_text  Tag_Number_final
0       detail disclosure indicator uibutton simple wa...                 8
1       hello world fail emulator follow hello world t...                 4
2       jshint throw possible strict violation line tr...                 3
3       programmatically bound column invisible try da...                 9
4       edittext get focus soft keyboard android home ...                 4
...                                                   ...               ...
188873  reload tab activity tab change reload activity...                 4
188874  learning index loop list contain list inner li...                 7
188875  module pattern | private access pass reference...                 3
188876  listener homebutton android   possible duplica...                 4
188877  php optimize multiple conditional loop multipl...                 2

[188878 rows x 2 columns]


In [35]:
stackExchange.dropna(inplace=True)
print(stackExchange)

                                             cleaned_text  Tag_Number_final
0       detail disclosure indicator uibutton simple wa...                 8
1       hello world fail emulator follow hello world t...                 4
2       jshint throw possible strict violation line tr...                 3
3       programmatically bound column invisible try da...                 9
4       edittext get focus soft keyboard android home ...                 4
...                                                   ...               ...
188873  reload tab activity tab change reload activity...                 4
188874  learning index loop list contain list inner li...                 7
188875  module pattern | private access pass reference...                 3
188876  listener homebutton android   possible duplica...                 4
188877  php optimize multiple conditional loop multipl...                 2

[188874 rows x 2 columns]


In [36]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80/20 split)
SExchangeTrain, test_val = train_test_split(stackExchange, test_size=0.2, random_state=42)

# Split train set into train and validation sets (50/50 split)
SExchangeTest, SExchangeVal = train_test_split(test_val, test_size=0.5, random_state=42)

# Print the shapes & percentage of split of the resulting datasets
print("Train set shape: ", SExchangeTrain.shape, "Split %: ", len(SExchangeTrain)/len(stackExchange))
print("Validation set shape: ", SExchangeVal.shape,"Split %: ", len(SExchangeVal)/len(stackExchange))
print("Test set shape: ", SExchangeTest.shape, "Split %: ", len(SExchangeTest)/len(stackExchange))

Train set shape:  (151099, 2) Split %:  0.7999989410930038
Validation set shape:  (18888, 2) Split %:  0.1000031767209886
Test set shape:  (18887, 2) Split %:  0.09999788218600761


In [37]:
X_SExchangeTrain = SExchangeTrain["cleaned_text"]
X_SExchangeTest = SExchangeTest["cleaned_text"]
X_SExchangeValid = SExchangeVal["cleaned_text"]

Y_SExchangeTrain = SExchangeTrain["Tag_Number_final"]
Y_SExchangeTest = SExchangeTest["Tag_Number_final"]
Y_SExchangeValid = SExchangeVal["Tag_Number_final"]

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install wandb -qq
import wandb
import numpy as np

# Create Vectors using Tfidf

In [12]:
#vectorizer = TfidfVectorizer(
 #   stop_words='english', min_df=4, max_features=5000).fit(X_SExchangeTrain)
#X_SEtrain_vec = vectorizer.transform(X_SExchangeTrain)
#X_SEvalid_vec = vectorizer.transform(X_SExchangeValid)
#X_SEtest_vec = vectorizer.transform(X_SExchangeTest)

#Create PyTorch Tensors

In [13]:
# Creating a tensors of data
#X_SEtrain_Tensor = torch.tensor(X_SEtrain_vec.toarray()).float()
#X_SEvalid_Tensor = torch.tensor(X_SEvalid_vec.toarray()).float()
#X_SEtest_Tensor = torch.tensor(X_SEtest_vec.toarray()).float()

#y_SEtrain_Tensor = torch.tensor(np.array(Y_SExchangeTrain)).long()
#y_SEvalid_Tensor = torch.tensor(np.array(Y_SExchangeValid)).long()
#y_SEtest_Tensor = torch.tensor(np.array(Y_SExchangeTest)).long()


#Create PyTorch Dataset

In [14]:
# Creating tensor dataset
#SETrain_set = torch.utils.data.TensorDataset(X_SEtrain_Tensor, y_SEtrain_Tensor)
#SETest_set = torch.utils.data.TensorDataset(X_SEtest_Tensor, y_SEtest_Tensor)
#SEValid_set = torch.utils.data.TensorDataset(X_SEvalid_Tensor, y_SEvalid_Tensor)

#Custom class

In [39]:
class CustomClass(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y 
  def __len__(self):
        return len(self.X)

  def __getitem__(self, idx):
    text = self.X.iloc[idx]
    labels = self.y.iloc[idx]
    tup = (labels, text)
    return tup

In [40]:
SEtrain_dataset = CustomClass(X_SExchangeTrain, Y_SExchangeTrain)
i=0
for i, (x,y) in enumerate(SEtrain_dataset):
  if(i<=10):
    print(i, x,y)
    i+=1
  else:
    break
SEtrain_dataset.__getitem__([2])
SEtrain_dataset[2]

0 7 python read ini file python want transform joomla ini language file sql joomla ini file actually miss section example translation rawconfigparser job demand section construct temp file dummy section name      fout = tempfile namedtemporaryfile(delete = true      fin = file(self._infilename r      fout.write("[all]\n      f              fout.write(f      config = configparser rawconfigparser(allow_no_value = true           c config.items("all              self._ini2sql(unicode(c[0]).upper unicode('de unicode(c[1][1:-1   def elegant solution tip pythonic
1 2 php data difference give fatal error date value come database want calculate difference today date database date date come database 2012 06 11 18:20:40   use code value      echo date('y m d h s      echo $ result['dt_pub_date   write code $ val = date('y m d h s ->diff($result['dt_pub_date   get error fatal error member function diff non object   thank
2 4 convert arraylist message convert arraylist message convert msg arraylist

(4,
 'convert arraylist message convert arraylist message convert msg arraylist arraylist extractdata = arraylist   opposite right message msg=(message arraylist  ')

In [41]:
SEtest_dataset = CustomClass(X_SExchangeTest, Y_SExchangeTest)
i=0
for i, (x,y) in enumerate(SEtest_dataset):
  if(i<=10):
    print(i, x,y)
    i+=1
  else:
    break

0 4 android tab support v4l2 usb host functionality need know name android tablet v4l2 support host functionality suppose plug usb camera tablet functionality thank lot advance
1 2 filename php $ str = /d file shop/7c0ea5b0b96a490a9c78255ddb559943.jpg   filename path thank lot
2 6 c++ ` my_map[i = ` std::map < int int > my_map my_map[0 =   my_map[0 0 1 undefined
3 0 code path return value method public bool userexistsactivedir     try            const int ads_uf_accountdisable = 0x00000002        directoryentry de = new directoryentry        de path = ldap://domainname        directorysearcher objadsearcher = new directorysearcher(de        de authenticationtype = authenticationtypes secure         objadsearcher searchroot = de        objadsearcher filter = samaccountname= + txtusername text +        searchresult result = objadsearcher findone         result tostring =                  int flag = convert toint32(results properties["useraccountcontrol"][0].tostre           //results pro

In [42]:
SEvalid_dataset = CustomClass(X_SExchangeValid, Y_SExchangeValid)
i=0
for i, (x,y) in enumerate(SEvalid_dataset):
  if(i<=10):
    print(i, x,y)
    i+=1

0 3 cross domain ajax activexobject ie activexobject object ie submit cross domain request
1 5 wrap script function professional approach right work little training code try code semi fixed page element result   $ document).ready(function      var elementposition = $ header").offset      var elementheight = $ header").outerheight      $ header").before("<div id='placeholder style='display height + elementheight + px'></div >      $ window).scroll(function          checkattached("#header            function checkattached(element          var windowdepth = $ window).scrolltop          windowdepth > =              $ element).addclass("attached              $ placeholder").css("display block                       $ element).removeclass("attache              $ placeholder").css("display                 thing happy solution like little flexible goal independent function allow address page element like attachthis(element   right have trouble scroll event inside function know good solution ove

#Create Vocab

In [43]:
def createVocab(Dataset):
  counter = Counter()
  for (label, line) in Dataset:
    counter.update(str(line).split())
  my_vocab = vocab(counter, min_freq=5)
  # get mapping of words to index
  my_vocab.get_stoi()
  # insert '<unk>' token to represent any unknown word
  my_vocab.insert_token('<unk>', 0)
  # set the default index to zero
  # thus any uknown word will be represented b index 0 or token '<unk>'
  my_vocab.set_default_index(0)
  return my_vocab

In [55]:
#global SETrain_vocab, SETest_vocab, SEValid_vocab
SETrain_vocab = createVocab(SEtrain_dataset)
SETest_vocab = createVocab(SEtest_dataset)
SEValid_vocab = createVocab(SEvalid_dataset)

In [21]:
#SEValid_vocab.get_stoi()

#Create collate_batch function

In [58]:
# Creating a lambda function objects that will be used to get the indices of words from vocab
def text_pipeline(x, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(x).split()]

In [57]:
def collate_batch(batch):
    labels, texts = zip(*batch)
    labels = torch.tensor(labels, dtype=torch.int64)
    #if 
    list_of_list_of_indices = [text_pipeline(text, SETrain_vocab) for text in texts]
    
    offsets = [0] + [len(i) for i in list_of_list_of_indices]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    texts = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])
    return labels, texts, offsets

#Create DataLoaders

In [59]:
#Initiale the batch size 


# create DataLoader now
torch.manual_seed(0)
batch_size = 256
SEtrain_loader = torch.utils.data.DataLoader(dataset=SEtrain_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=lambda batch: collate_batch(batch, SETrain_vocab),
                                           )

SEvalid_loader = torch.utils.data.DataLoader(dataset=SEvalid_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=lambda batch: collate_batch(batch, SEValid_vocab),
                                           )
SEtest_loader = torch.utils.data.DataLoader(dataset=SEtest_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=lambda batch: collate_batch(batch, SETest_vocab),
                                           )


In [61]:
torch.utils.data.DataLoader?

In [60]:
for label, text, offsets in SEtrain_loader:
  print(label, text, offsets)

Streaming output truncated to the last 5000 lines.
          765,  1114,  1144,  1338,  1379,  1405,  1470,  1503,  1562,  1621,
         1708,  1723,  1878,  2362,  2392,  2426,  2473,  2498,  2568,  2690,
         2719,  2754,  2825,  2845,  2911,  2935,  2982,  2996,  3009,  3044,
         3483,  3527,  3566,  3612,  3681,  3718,  3735,  3747,  3768,  3803,
         3817,  3889,  3955,  4018,  4083,  4171,  4209,  4246,  4315,  4329,
         4383,  4419,  4442,  4471,  4495,  4513,  4561,  4594,  4656,  4692,
         4761,  4798,  4983,  5016,  5076,  5134,  5202,  5330,  5401,  5475,
         5504,  5538,  5611,  5656,  5760,  6076,  6090,  6263,  6351,  6383,
         6405,  6450,  6635,  6716,  6761,  6806,  6839,  6852,  6896,  6927,
         6977,  7205,  7246,  7283,  7409,  7425,  7440,  7457,  7509,  7575,
         7603,  7662,  7703,  7926,  7944,  7988,  8049,  8071,  8139,  8175,
         8278,  8296,  8327,  8375,  8412,  8484,  8511,  8580,  8663,  8690,
         8729

#Model

In [26]:
import torch.nn as nn

class MLPCustom(nn.Module):
    def __init__(self, vocab_size, embed_dim=300, n_hidden1=200, n_hidden2=100, n_outs=1, dropout1=0.5, dropout2=0.5):
        super().__init__()

        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)

        self.hidden_layer1 = nn.Linear(embed_dim, n_hidden1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout1)
        self.batchnorm1 = nn.BatchNorm1d(n_hidden1)

        self.hidden_layer2 = nn.Linear(n_hidden1, n_hidden2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=dropout2)
        self.batchnorm2 = nn.BatchNorm1d(n_hidden2)

        self.output_layer = nn.Linear(n_hidden2, n_outs)

    def forward(self, input, offsets):
        # input: tensor of shape (batch_size, seq_len)
        # offsets: tensor of shape (batch_size,) containing the starting indices of each sequence in the input tensor

        embedded = self.embedding(input, offsets)
        # embedded: tensor of shape (batch_size, embed_dim)

        h1 = self.hidden_layer1(embedded)
        h1 = self.relu1(h1)
        h1 = self.dropout1(h1)
        h1 = self.batchnorm1(h1)
        # h1: tensor of shape (batch_size, n_hidden1)

        h2 = self.hidden_layer2(h1)
        h2 = self.relu2(h2)
        h2 = self.dropout2(h2)
        h2 = self.batchnorm2(h2)
        # h2: tensor of shape (batch_size, n_hidden2)

        ypred = self.output_layer(h2)
        # ypred: tensor of shape (batch_size, n_outs)

        return ypred

In [27]:
# Initialize a new project
wandb.init(name="imdb_shallow_non_seq", project='nlps23_L7')
#20efc82737d336646269a333588441a0c04c4c0d        --API Key
# Initialize number of epochs, learning rate and batch size
learning_rate = 0.2
wandb.log({'learning_rate': learning_rate})
epochs = 20

# device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#n_inputs = train_set.tensors[0].shape[1]

model = MLPCustom(vocab_size, embed_dim=300, n_hidden1=200, n_hidden2=100, n_outs=1, dropout1=0.5, dropout2=0.5)

#model = MLPCustom(n_inputs, n_hidden1, n_hidden2,n_outputs, non_linearity, bias=True)

# loss_function
loss_function = nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), learning_rate)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


NameError: ignored

#Custom class to train, predict accuracy

In [ ]:
import torch
import wandb

def step(inputs, targets, model, device, loss_function=None, optimizer=None):
    """ 
    Helper function for processing a batch of data.
    Input: inputs, targets, model, loss function, optimizer (optional)
    Output: loss, predicted outputs, and number of correct predictions
    """
    model = model.to(device)
    # Move inputs and targets to the device
    inputs = inputs.to(device)
    targets = targets.to(device)

    # Step 1: Forward Pass: Compute model's predictions
    outputs = model(inputs)

    # Step 2: Compute loss
    if loss_function:
        loss = loss_function(outputs, targets)

    # Step 3: Compute accuracy
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == targets).sum().item()

    # Step 4: Backward pass and update parameters if optimizer is given
    if optimizer is not None:
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if loss_function:
        return loss, outputs, correct
    else:
        return outputs, correct


def train_epoch(train_loader, model, device, loss_function, optimizer):
    """ 
    Function for training the model for one epoch.
    Input: iterator for train dataset, loss function, model, optimizer (optional)
    Output: train loss and accuracy for the epoch
    """
    model = model.to(device)
    # Set model to training mode
    model.train()

    # Initialize train_loss and train_correct for the epoch
    running_train_loss = 0
    running_train_correct = 0

    # Iterate over batches from the dataset using train_loader
    for inputs, targets in train_loader:
        # Process the batch
        loss, _, correct = step(inputs, targets, model, device,
                                loss_function, optimizer)

        # Add batch loss and accuracy to running total
        running_train_loss += loss.item()
        running_train_correct += correct

    # Calculate mean train loss and accuracy for the epoch
    train_loss = running_train_loss / len(train_loader)
    train_acc = running_train_correct / len(train_loader.dataset)

    return train_loss, train_acc


def val_epoch(valid_loader, model, device, loss_function):
    """ 
    Function for validating the model for one epoch.
    Input: iterator for validation dataset, loss function, model
    Output: validation loss and accuracy for the epoch
    """
    model = model.to(device)
    # Set model to evaluation mode
    model.eval()

    # Initialize val_loss and val_correct for the epoch
    running_val_loss = 0
    running_val_correct = 0

    # Iterate over batches from the dataset using valid_loader
    with torch.no_grad():
        for inputs, targets in valid_loader:
            # Process the batch
            loss, _, correct = step(
                inputs, targets, model, device, loss_function, optimizer=None)

            # Add batch loss and accuracy to running total
            running_val_loss += loss.item()
            running_val_correct += correct

    # Calculate mean validation loss and accuracy for the epoch
    val_loss = running_val_loss / len(valid_loader)
    val_acc = running_val_correct / len(valid_loader.dataset)

    return val_loss, val_acc


def train(train_loader, valid_loader, model, optimizer, loss_function, epochs, device):
    """ 
    Function for training the model and plotting the graph for train & validation loss vs epoch.
    Input: iterator for train and validation datasets, model, optimizer, loss function, number of epochs, device
    Output: train loss and accuracy history, validation loss and accuracy history
    """

    # Create lists to store train and val loss and accuracy at each epoch
    train_loss_history = []
    valid_loss_history = []
    train_acc_history = []
    valid_acc_history = []

    # Iterate for the given number of epochs
    for epoch in range(epochs):

        # Train the model for one epoch and get train loss and accuracy
        train_loss, train_acc = train_epoch(
            train_loader, model, device, loss_function, optimizer)

        # Validate the model and get validation loss and accuracy
        valid_loss, valid_acc = val_epoch(
            valid_loader, model, device, loss_function)

        # Append the loss and accuracy to their respective lists
        train_loss_history.append(train_loss)
        train_acc_history.append(train_acc)
        valid_loss_history.append(valid_loss)
        valid_acc_history.append(valid_acc)

        # Log the train and valid loss and accuracy to WandB
        wandb.log({"Train Loss": train_loss, "Train Accuracy": train_acc})
        wandb.log({"Valid Loss": valid_loss, "Valid Accuracy": valid_acc})

        # Print the train and valid loss and accuracy for the epoch
        print(f"Epoch {epoch+1}/{epochs}")
        print(
            f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc*100:.2f}%")
        print(
            f"Valid Loss: {valid_loss:.4f} | Valid Accuracy: {valid_acc*100:.2f}%")
        print()

    return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history


def get_acc_pred(data_loader, model, device):
    """ 
    Function to get predictions and accuracy for a given data using a trained model
    Input: data iterator, model, device
    Output: predictions and accuracy for the given dataset
    """
    model = model.to(device)
    # Set model to evaluation mode
    model.eval()

    # Create empty tensors to store predictions and actual labels
    predictions = torch.Tensor().to(device)
    y = torch.Tensor().to(device)

    # Iterate over batches from data iterator
    with torch.no_grad():
        for inputs, targets in data_loader:
            # Process the batch to get the loss, outputs, and correct predictions
            outputs, _ = step(inputs, targets, model,
                              device, loss_function=None, optimizer=None)

            # Choose the label with maximum probability
            _, predicted = torch.max(outputs.data, 1)

            # Add the predicted labels and actual labels to their respective tensors
            predictions = torch.cat((predictions, predicted))
            y = torch.cat((y, targets.to(device)))

    # Calculate accuracy by comparing the predicted and actual labels
    accuracy = (predictions == y).float().mean()

    # Return tuple containing predictions and accuracy
    return predictions, accuracy

In [ ]:
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train(SEtrain_loader,
                                                                                     SEvalid_loader,
                                                                                     model,
                                                                                     optimizer,
                                                                                     loss_function,
                                                                                     epochs,
                                                                                     device)


Epoch 1/20
Train Loss: 0.6919 | Train Accuracy: 52.45%
Valid Loss: 0.6896 | Valid Accuracy: 49.70%

Epoch 2/20
Train Loss: 0.6835 | Train Accuracy: 58.69%
Valid Loss: 0.6751 | Valid Accuracy: 61.82%

Epoch 3/20
Train Loss: 0.6504 | Train Accuracy: 75.75%
Valid Loss: 0.6319 | Valid Accuracy: 59.64%

Epoch 4/20
Train Loss: 0.5724 | Train Accuracy: 74.48%
Valid Loss: 0.5289 | Valid Accuracy: 73.22%

Epoch 5/20
Train Loss: 0.5281 | Train Accuracy: 70.80%
Valid Loss: 0.4641 | Valid Accuracy: 79.04%

Epoch 6/20
Train Loss: 0.4960 | Train Accuracy: 73.89%
Valid Loss: 0.5070 | Valid Accuracy: 72.46%

Epoch 7/20
Train Loss: 0.4478 | Train Accuracy: 77.87%
Valid Loss: 0.6597 | Valid Accuracy: 65.68%

Epoch 8/20
Train Loss: 0.4182 | Train Accuracy: 80.50%
Valid Loss: 0.4763 | Valid Accuracy: 75.92%

Epoch 9/20
Train Loss: 0.4029 | Train Accuracy: 81.41%
Valid Loss: 0.4763 | Valid Accuracy: 76.32%

Epoch 10/20
Train Loss: 0.3696 | Train Accuracy: 83.31%
Valid Loss: 0.3537 | Valid Accuracy: 84.50%
